In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd ..
%cd /content/drive/MyDrive

Mounted at /content/drive
/
/content/drive/MyDrive


In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the weather time series data from a CSV file
df = pd.read_csv('/content/drive/MyDrive/Suny Buffalo/Sem 1 fall/Data intensive computing/Project/max_planck_weather_ts.csv')

# Select every 6th row starting from the 6th row (index 5)
df = df[5::6]

# Convert 'Date Time' column to datetime format and set it as the time series index
time_series = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

# Handle missing values in wind velocity data
wind_velocity = df['wv (m/s)']
missing_wind = wind_velocity == -9999.0
wind_velocity[missing_wind] = 0.0

# Handle missing values in max wind velocity data
max_wind_velocity = df['max. wv (m/s)']
missing_max_wind = max_wind_velocity == -9999.0
max_wind_velocity[missing_max_wind] = 0.0

# Calculate the minimum wind velocity
min_wind_velocity = df['wv (m/s)'].min()

# Extract wind velocity and max wind velocity columns
wind_velocity = df['wv (m/s)']
missing_wind = wind_velocity == -9999.0
wind_velocity[missing_wind] = 0.0

max_wind_velocity = df['max. wv (m/s)']
missing_max_wind = max_wind_velocity == -9999.0
max_wind_velocity[missing_max_wind] = 0.0

min_wind_velocity = df['wv (m/s)'].min()

# Extract and convert wind direction data to radians
wind_direction_degrees = df.pop('wd (deg)')
wind_direction_radians = wind_direction_degrees * np.pi / 180

# Calculate the wind vector components and max wind vector components
df['Wind_X'] = wind_velocity * np.cos(wind_direction_radians)
df['Wind_Y'] = wind_velocity * np.sin(wind_direction_radians)

df['Max_Wind_X'] = max_wind_velocity * np.cos(wind_direction_radians)
df['Max_Wind_Y'] = max_wind_velocity * np.sin(wind_direction_radians)

# Split the data into training, validation, and test sets
column_indices = {name: i for i, name in enumerate(df.columns)}
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
test_df = df[int(n * 0.99):]

# Normalize the data using mean and standard deviation from the training set
num_features = df.shape[1]
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

# Reassign column indices for clarity
column_indices = {name: i for i, name in enumerate(df.columns)}
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
#test_df = df[int(n * 0.99):]
test_df = df[n-1:]

# Normalize the test set using mean and standard deviation from the training set
num_features = df.shape[1]
test_df = (test_df - train_mean) / train_std

# Instructions for running the code with different datasets:
# 1. Change the file path in the pd.read_csv() function to the location of your dataset.
# 2. Ensure that the dataset has the required columns ('Date Time', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)').
# 3. Execute the code step by step or as a whole, depending on your preference.
# 4. Monitor the comments for each section to understand the data processing steps.


In [ ]:
display(test_df)

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),Wind_X,Wind_Y,Max_Wind_X,Max_Wind_Y
420545,1000.11,-3.93,269.23,-8.09,72.6,4.56,3.31,1.25,2.06,3.31,1292.41,-0.516998,-0.215205,-0.92321,-0.384295


##Making a window

In [ ]:
# Import necessary libraries
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define a class for managing windowed time series data
class WindowGenerator:
    def __init__(self, input_width, label_width, shift,
                 train_df, val_df, test_df,
                 label_columns=None):
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

# Create instances of the WindowGenerator class with specific parameters
w1 = WindowGenerator(input_width=24, label_width=1, shift=24,
                     train_df=train_df, val_df=val_df, test_df=test_df,
                     label_columns=['T (degC)'])
w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     train_df=train_df, val_df=val_df, test_df=test_df,
                     label_columns=['T (degC)'])

# Function to split a window into inputs and labels
def split_window(self, features):
    inputs = features[:, :, self.input_slice]
    labels = features[:, :, self.labels_slice]
    if self.label_columns is not None:
        labels = torch.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], dim=-1)
    return inputs, labels

# Add the split_window function to the WindowGenerator class
WindowGenerator.split_window = split_window

# Example usage of the split_window function
example_window = torch.stack([torch.tensor(train_df[:w2.total_window_size].values),
                               torch.tensor(train_df[100:100 + w2.total_window_size].values),
                               torch.tensor(train_df[200:200 + w2.total_window_size].values)])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

# Function to create a PyTorch dataset from the windowed time series data
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    dataset = CustomTimeSeriesDataset(data, self.total_window_size)
    return DataLoader(dataset, batch_size=32, shuffle=True)

# Add the make_dataset function to the WindowGenerator class
WindowGenerator.make_dataset = make_dataset

# Properties to access training, validation, and test datasets
@property
def train(self):
    return self.make_dataset(self.train_df.values)

@property
def val(self):
    return self.make_dataset(self.val_df.values)

@property
def test(self):
    return self.make_dataset(self.test_df.values)

# Property to access an example batch from the training dataset
@property
def example(self):
    result = getattr(self, '_example', None)
    if result is None:
        result = next(iter(self.train))
        self._example = result
    return result

# Add properties to the WindowGenerator class
WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

# Define a custom PyTorch dataset for time series data
class CustomTimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length + 1

    def __getitem__(self, idx):
        return self.data[idx:idx + self.sequence_length]

# Example usage of the CustomTimeSeriesDataset
for example_data in w2.train:
    example_data = example_data.view(-1, example_data.shape[-1])
    example_inputs, example_labels = example_data[:, :-1], example_data[:, -1]
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

# Instructions for running the code with different datasets:
# 1. Create or load your time series dataset and ensure it has similar column names.
# 2. Create an instance of the WindowGenerator class with the appropriate parameters for your dataset.
# 3. Access training, validation, and test datasets using the properties train, val, and test, respectively.
# 4. Utilize the example property to get an example batch from the training dataset for testing and debugging.
# 5. If needed, modify the class properties and methods to adapt to the specific characteristics of your dataset.
# 6. Execute the code and monitor the printed shapes and information for understanding the data processing steps.


All shapes are: (batch, time, features)
Window shape: torch.Size([3, 7, 15])
Inputs shape: torch.Size([3, 7, 6])
Labels shape: torch.Size([3, 7, 1])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): torch.Size([224, 14])
Labels shape (batch, time, features): torch.Size([224])
Inputs shape (batch, time, features): t

In [ ]:
display(train_df.columns)

Index(['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
       'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wind_X', 'Wind_Y', 'Max_Wind_X',
       'Max_Wind_Y'],
      dtype='object')

In [ ]:
# import torch
# import xgboost as xgb
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# features = ['p (mbar)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
#             'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
#             'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wind_X', 'Wind_Y', 'Max_Wind_X',
#             'Max_Wind_Y']
# target = 'T (degC)'

# X_train, y_train = train_df[features], train_df[target]
# X_val, y_val = val_df[features], val_df[target]
# X_test, y_test = test_df[features], test_df[target]

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dval = xgb.DMatrix(X_val, label=y_val)
# dtest = xgb.DMatrix(X_test, label=y_test)


In [ ]:
X_test.to_csv('/content/input_test.csv')

##Logistic Regression

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# Assuming df is your time series dataframe
# Split the data into training, validation, and test sets
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
test_df = df[int(n * 0.9):]

# Define target and feature columns
target_column = 'T (degC)'
feature_columns = ['p (mbar)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
                   'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)',
                   'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)',
                   'Wind_X', 'Wind_Y', 'Max_Wind_X', 'Max_Wind_Y']

# Extract target and feature values for training, validation, and test sets
train_target = train_df[target_column].values
val_target = val_df[target_column].values
test_target = test_df[target_column].values

train_features = train_df[feature_columns].values
val_features = val_df[feature_columns].values
test_features = test_df[feature_columns].values

# Standardize features using StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

# Convert data to PyTorch tensors
train_features_tensor = torch.FloatTensor(train_features)
val_features_tensor = torch.FloatTensor(val_features)
test_features_tensor = torch.FloatTensor(test_features)

train_target_tensor = torch.FloatTensor(train_target)
val_target_tensor = torch.FloatTensor(val_target)
test_target_tensor = torch.FloatTensor(test_target)

# Set batch size and create DataLoader for training set
batch_size = 32
train_dataset = TensorDataset(train_features_tensor, train_target_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define a simple linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        out = self.linear(x)
        return out

# Instantiate the model
input_size = len(feature_columns)
model = LinearRegressionModel(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Set the number of training epochs
num_epochs = 100

# List to store training losses for plotting or analysis
train_losses = []

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

# Evaluate the model on the validation set
with torch.no_grad():
    val_outputs = model(val_features_tensor)
    val_loss = criterion(val_outputs, val_target_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')

# Evaluate the model on the test set
test_outputs = model(test_features_tensor)
test_loss = criterion(test_outputs, test_target_tensor)
print(f'Test Loss: {test_loss.item():.4f}')

# Instructions for running the code with different datasets:
# 1. Change the value of 'df' to your desired time series dataframe.
# 2. Adjust the 'target_column' and 'feature_columns' according to your dataset columns.
# 3. Ensure that the dataset is split into training, validation, and test sets.
# 4. Execute the code step by step or as a whole, depending on your preference.
# 5. Monitor the printed validation and test losses for model evaluation.


In [ ]:
# Create a figure for plotting
plt.figure(figsize=(8, 6))

# Plot the training loss over iterations
plt.plot(range(len(train_losses)), train_losses, label='Training Loss')

# Set labels for the x-axis and y-axis
plt.xlabel('Iterations')
plt.ylabel('Loss')

# Set the title of the plot
plt.title('Training Loss over Iterations')

# Display the legend to identify the plotted curve
plt.legend()

# Show the plot
plt.show()


##LSTM

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Set the number of training epochs
num_epochs = 50

# Define the LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

# Standardize the data using StandardScaler
scaler = StandardScaler()
train_data = scaler.fit_transform(train_df.values)
val_data = scaler.transform(val_df.values)
test_data = scaler.transform(test_df.values)

# Function to create sequences for LSTM training
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
    return np.array(sequences)

# Define the sequence length and create sequences for training, validation, and test
sequence_length = 10
train_sequences = create_sequences(train_data, sequence_length)
val_sequences = create_sequences(val_data, sequence_length)
test_sequences = create_sequences(test_data, sequence_length)

# Convert sequences to PyTorch tensors
train_sequences = torch.tensor(train_sequences, dtype=torch.float32)
val_sequences = torch.tensor(val_sequences, dtype=torch.float32)
test_sequences = torch.tensor(test_sequences, dtype=torch.float32)

# Create DataLoader for training, validation, and test sets
train_loader = torch.utils.data.DataLoader(train_sequences, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_sequences, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_sequences, batch_size=32)

# Define input size, hidden size, and number of layers for the LSTM model
input_size = len(df.columns)
hidden_size = 32
num_layers = 1

# Instantiate the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# List to store training losses for plotting or analysis
train_losses = []

# Training loop
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

# Set the model to evaluation mode for validation and test
model.eval()

# Validation evaluation
val_losses = []
with torch.no_grad():
    for batch in val_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        val_loss = criterion(output, target)
        val_losses.append(val_loss.item())

val_loss = np.mean(val_losses)
print('Validation Loss:', val_loss)

# Test evaluation
test_losses = []
with torch.no_grad():
    for batch in test_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        test_loss = criterion(output, target)
        test_losses.append(test_loss.item())

test_loss = np.mean(test_losses)
print('Test Loss:', test_loss)

# Instructions for running the code with different datasets:
# 1. Replace train_df, val_df, and test_df with your desired time series dataframes.
# 2. Adjust the 'input_size' according to the number of features in your dataset.
# 3. Modify 'target_column' and 'feature_columns' if your dataset columns differ.
# 4. Ensure that the data is split into training, validation, and test sets.
# 5. Execute the code step by step or as a whole, depending on your preference.
# 6. Monitor the printed validation and test losses for model evaluation.


In [ ]:
# Plot training loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

# Plot validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(range(num_epochs), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation Loss over Epochs')
plt.legend()
plt.show()

# Plot test loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(range(num_epochs), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Test Loss over Epochs')
plt.legend()
plt.show()


##Bi-LSTM

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Define the Bidirectional LSTM model class
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTMModel, self).__init__()
        # Bidirectional LSTM layer
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # Linear layer to map the concatenated hidden states to the output
        self.linear = nn.Linear(2 * hidden_size, 1)

    def forward(self, x):
        # Bidirectional LSTM forward pass
        out, _ = self.bilstm(x)
        # Use only the last time step and concatenate both directions
        out = self.linear(out[:, -1, :])
        return out

# Standardize the data using StandardScaler
scaler = StandardScaler()
train_data = scaler.fit_transform(train_df.values)
val_data = scaler.transform(val_df.values)
test_data = scaler.transform(test_df.values)

# Function to create sequences for LSTM training
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
    return np.array(sequences)

# Define the sequence length and create sequences for training, validation, and test
sequence_length = 8
train_sequences = create_sequences(train_data, sequence_length)
val_sequences = create_sequences(val_data, sequence_length)
test_sequences = create_sequences(test_data, sequence_length)

# Convert sequences to PyTorch tensors
train_sequences = torch.tensor(train_sequences, dtype=torch.float32)
val_sequences = torch.tensor(val_sequences, dtype=torch.float32)
test_sequences = torch.tensor(test_sequences, dtype=torch.float32)

# Create DataLoader for training, validation, and test sets
train_loader = torch.utils.data.DataLoader(train_sequences, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_sequences, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_sequences, batch_size=32)

# Define input size, hidden size, and number of layers for the Bidirectional LSTM model
input_size = len(df.columns)
hidden_size = 32
num_layers = 1

# Instantiate the Bidirectional LSTM model
model = BiLSTMModel(input_size, hidden_size, num_layers)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the number of training epochs
num_epochs = 50
train_losses = []

# Training loop
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

# Set the model to evaluation mode for validation and test
model.eval()

# Validation evaluation
val_losses = []
with torch.no_grad():
    for batch in val_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        val_loss = criterion(output, target)
        val_losses.append(val_loss.item())

val_loss = np.mean(val_losses)
print('Validation Loss:', val_loss)

# Test evaluation
test_losses = []
with torch.no_grad():
    for batch in test_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        test_loss = criterion(output, target)
        test_losses.append(test_loss.item())

test_loss = np.mean(test_losses)
print('Test Loss:', test_loss)


In [ ]:
# Plot training loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

# Set the model to evaluation mode for validation
model.eval()
val_predictions = []
val_targets = []

# Generate predictions and targets for the validation set
with torch.no_grad():
    for batch in val_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        val_predictions.extend(output.numpy())
        val_targets.extend(target.numpy())

# Scatter plot for validation set - Predicted vs Actual
plt.figure(figsize=(8, 6))
plt.scatter(val_targets, val_predictions, alpha=0.5)
plt.title('Validation Set - Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

# Generate predictions and targets for the test set
test_predictions = []
test_targets = []

with torch.no_grad():
    for batch in test_loader:
        output = model(batch[:, :, :-1])
        target = batch[:, -1, -2].unsqueeze(1)
        test_predictions.extend(output.numpy())
        test_targets.extend(target.numpy())

# Scatter plot for test set - Predicted vs Actual
plt.figure(figsize=(8, 6))
plt.scatter(test_targets, test_predictions, alpha=0.5)
plt.title('Test Set - Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()


##ARIMA

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Suny Buffalo/Sem 1 fall/Data intensive computing/Project/max_planck_weather_ts.csv')

# Select every 6th row starting from the 5th row to downsample the data
df = df[5::6]

# Handle missing values in 'wv (m/s)' by replacing -9999.0 with 0.0
wind_velocity = df['wv (m/s)']
missing_wind = wind_velocity == -9999.0
wind_velocity[missing_wind] = 0.0

# Handle missing values in 'max. wv (m/s)' by replacing -9999.0 with 0.0
max_wind_velocity = df['max. wv (m/s)']
missing_max_wind = max_wind_velocity == -9999.0
max_wind_velocity[missing_max_wind] = 0.0

# Calculate the minimum wind velocity
min_wind_velocity = df['wv (m/s)'].min()

# Duplicate code block, remove the redundant part
# ...

# Pop 'wv (m/s)' and 'max. wv (m/s)' columns and store them in variables
wind_velocity = df.pop('wv (m/s)')
max_wind_velocity = df.pop('max. wv (m/s)')

# Convert wind direction from degrees to radians
wind_direction_degrees = df.pop('wd (deg)')
wind_direction_radians = wind_direction_degrees * np.pi / 180

# Calculate wind components in X and Y directions
df['Wind_X'] = wind_velocity * np.cos(wind_direction_radians)
df['Wind_Y'] = wind_velocity * np.sin(wind_direction_radians)

df['Max_Wind_X'] = max_wind_velocity * np.cos(wind_direction_radians)
df['Max_Wind_Y'] = max_wind_velocity * np.sin(wind_direction_radians)

# Create a dictionary to map column names to their indices
column_indices = {name: i for i, name in enumerate(df.columns)}

# Split the DataFrame into training, validation, and test sets
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
test_df = df[int(n * 0.9):]

# Get the number of features in the DataFrame
num_features = df.shape[1]

# Standardize the data by subtracting the mean and dividing by the standard deviation
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

# Return the processed training DataFrame
train_df

# Instructions for running the code with different datasets:

# Data Input: Update the file path in the pd.read_csv function to point to your desired CSV file containing weather time series data.

# Data Processing: The code processes the dataset by handling missing values, converting wind direction, and splitting it into training, validation, and test sets.

# Run the Code: Execute the code in a Python environment such as Jupyter Notebook or any other Python IDE.


<ipython-input-11-900e6cb32540>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_velocity[missing_wind] = 0.0
<ipython-input-11-900e6cb32540>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_wind_velocity[missing_max_wind] = 0.0
<ipython-input-11-900e6cb32540>:49: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_mean = train_df.mean()
<ipython-input-11-900e6cb32540>:50: FutureW

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),Wind_X,Wind_Y,Max_Wind_X,Max_Wind_Y
5,01.01.2009 01:00:00,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,-0.204862,-0.046168,-0.614587,-0.138503
11,01.01.2009 02:00:00,996.62,-8.88,264.54,-9.77,93.2,3.12,2.90,0.21,1.81,2.91,1312.25,-0.245971,-0.044701,-0.619848,-0.112645
17,01.01.2009 03:00:00,996.84,-8.81,264.59,-9.66,93.5,3.13,2.93,0.20,1.83,2.94,1312.18,-0.175527,0.039879,-0.614344,0.139576
23,01.01.2009 04:00:00,996.99,-9.05,264.34,-10.02,92.6,3.07,2.85,0.23,1.78,2.85,1313.61,-0.050000,-0.086603,-0.190000,-0.329090
29,01.01.2009 05:00:00,997.46,-9.63,263.72,-10.65,92.2,2.94,2.71,0.23,1.69,2.71,1317.19,-0.368202,0.156292,-0.810044,0.343843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294353,04.08.2014 21:10:00,990.06,16.94,290.93,15.56,91.6,19.33,17.71,1.62,11.20,17.89,1180.85,-0.929778,-2.746910,-1.449171,-4.281390
294359,04.08.2014 22:10:00,990.87,16.72,290.64,14.40,86.2,19.07,16.43,2.63,10.38,16.59,1183.30,-2.254649,-1.392500,-3.165017,-1.954755
294365,04.08.2014 23:10:00,990.46,15.52,289.47,14.54,93.9,17.66,16.58,1.08,10.48,16.74,1187.66,-0.656072,0.616092,-0.918500,0.862529
294371,05.08.2014 00:10:00,990.43,15.26,289.21,14.45,94.9,17.37,16.48,0.89,10.42,16.64,1188.74,-0.565397,-0.551748,-1.259619,-1.229211


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Convert 'Date Time' to datetime format and set it as the index
df['Date'] = pd.to_datetime(df['Date Time'])
df.set_index('Date', inplace=True)

# Specify the target column for forecasting
target_column = 'T (degC)'

# Split the dataset into training, validation, and test sets
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
test_df = df[int(n * 0.9):]

# Choose the order for the ARIMA model (p, d, q)
chosen_order = (2, 1, 1)

# Create an ARIMA model with the specified order and fit it to the training data
model = ARIMA(train_df[target_column], order=chosen_order)
model_fit = model.fit()

# Forecast the values for the test set
forecast = model_fit.forecast(steps=len(test_df))

# Calculate the Mean Squared Error (MSE) between the actual and forecasted values
mse = mean_squared_error(test_df[target_column], forecast)
print(f"Test Mean Squared Error: {mse}")

# Plot the actual and predicted values
plt.figure(figsize=(12, 6))
plt.plot(test_df.index, test_df[target_column], label='Actual')
plt.plot(test_df.index, forecast, label='Predicted', color='red')
plt.xlabel('Date')
plt.ylabel(target_column)
plt.legend()
plt.show()


##KNN

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Specify the target column and feature columns
target_column = 'T (degC)'
feature_columns = ['p (mbar)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
                   'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)',
                   'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)',
                   'Wind_X', 'Wind_Y', 'Max_Wind_X', 'Max_Wind_Y']

# Extract target and features for training, validation, and test sets
train_target = train_df[target_column].values
val_target = val_df[target_column].values
test_target = test_df[target_column].values

train_features = train_df[feature_columns].values
val_features = val_df[feature_columns].values
test_features = test_df[feature_columns].values

# Standardize features using StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

# Set the number of neighbors for KNeighborsRegressor
k_neighbors = 5

# Create and fit the KNeighborsRegressor model
knn_model = KNeighborsRegressor(n_neighbors=k_neighbors)
knn_model.fit(train_features, train_target)

# Make predictions on validation and test sets
val_predictions = knn_model.predict(val_features)
test_predictions = knn_model.predict(test_features)

# Calculate RMSE and R-squared for validation set
val_rmse = np.sqrt(mean_squared_error(val_target, val_predictions))
val_r2 = r2_score(val_target, val_predictions)

# Calculate RMSE and R-squared for test set
test_rmse = np.sqrt(mean_squared_error(test_target, test_predictions))
test_r2 = r2_score(test_target, test_predictions)

# Print results
print(f'Validation RMSE: {val_rmse:.4f}')
print(f'Test RMSE: {test_rmse:.4f}')
print(f'Validation R-squared: {val_r2:.4f}')
print(f'Test R-squared: {test_r2:.4f}')


In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt

# Scatter plot for Validation Set - Predicted vs Actual
plt.figure(figsize=(10, 6))
plt.scatter(val_target, val_predictions, alpha=0.5)
plt.title('Validation Set - Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

# Scatter plot for Test Set - Predicted vs Actual
plt.figure(figsize=(10, 6))
plt.scatter(test_target, test_predictions, alpha=0.5)
plt.title('Test Set - Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

# Calculate and plot residuals vs predicted values for the Validation Set
val_residuals = val_target - val_predictions
plt.figure(figsize=(10, 6))
plt.scatter(val_predictions, val_residuals, alpha=0.5)
plt.title('Validation Set - Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)  # Add a horizontal line at y=0 for reference
plt.show()

# Calculate and plot residuals vs predicted values for the Test Set
test_residuals = test_target - test_predictions
plt.figure(figsize=(10, 6))
plt.scatter(test_predictions, test_residuals, alpha=0.5)
plt.title('Test Set - Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)  # Add a horizontal line at y=0 for reference
plt.show()


##XGBoost

In [ ]:
!pip install xgboost

In [ ]:
import torch
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Define features and target variable
features = ['p (mbar)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
            'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
            'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wind_X', 'Wind_Y', 'Max_Wind_X',
            'Max_Wind_Y']
target = 'T (degC)'

# Split the data into training, validation, and test sets
X_train, y_train = train_df[features], train_df[target]
X_val, y_val = val_df[features], val_df[target]
X_test, y_test = test_df[features], test_df[target]

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'booster': 'gbtree',
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Train the XGBoost model
bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtrain, 'train'), (dval, 'val')], early_stopping_rounds=10)

# Make predictions on the test set
y_pred = bst.predict(dtest)

# Convert predicted values to labels
y_pred_labels = torch.round(torch.tensor(y_pred))

# Print the test Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Test Mean Squared Error: {mse}')

# Make predictions on the training set
y_train_pred = bst.predict(dtrain)
y_train_pred_labels = torch.round(torch.tensor(y_train_pred))

# Print the training Mean Squared Error
mse = mean_squared_error(y_train, y_train_pred_labels)
print(f'Training Mean Squared Error: {mse}')

# Make predictions on the validation set
y_val_pred = bst.predict(dval)
y_val_pred_labels = torch.round(torch.tensor(y_val_pred))

# Print the validation Mean Squared Error
mse = mean_squared_error(y_val, y_val_pred_labels)
print(f'Validation Mean Squared Error: {mse}')

# Print the predicted values on the test set
print(y_pred)


[0]	train-rmse:7.79257	val-rmse:6.89421
[1]	train-rmse:7.01635	val-rmse:6.20865
[2]	train-rmse:6.31737	val-rmse:5.59177
[3]	train-rmse:5.68807	val-rmse:5.03585


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:40:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[4]	train-rmse:5.12171	val-rmse:4.53612
[5]	train-rmse:4.61168	val-rmse:4.08627
[6]	train-rmse:4.15262	val-rmse:3.68027
[7]	train-rmse:3.73935	val-rmse:3.31612
[8]	train-rmse:3.36740	val-rmse:2.98816
[9]	train-rmse:3.03255	val-rmse:2.69246
[10]	train-rmse:2.73117	val-rmse:2.42658
[11]	train-rmse:2.45992	val-rmse:2.18742
[12]	train-rmse:2.21571	val-rmse:1.97168
[13]	train-rmse:1.99604	val-rmse:1.77774
[14]	train-rmse:1.79829	val-rmse:1.60350
[15]	train-rmse:1.62036	val-rmse:1.44659
[16]	train-rmse:1.46023	val-rmse:1.30551
[17]	train-rmse:1.31626	val-rmse:1.17888
[18]	train-rmse:1.18668	val-rmse:1.06472
[19]	train-rmse:1.07028	val-rmse:0.96195
[20]	train-rmse:0.96539	val-rmse:0.87007
[21]	train-rmse:0.87127	val-rmse:0.78727
[22]	train-rmse:0.78654	val-rmse:0.71307
[23]	train-rmse:0.71066	val-rmse:0.64605
[24]	train-rmse:0.64251	val-rmse:0.58599
[25]	train-rmse:0.58122	val-rmse:0.53258
[26]	train-rmse:0.52617	val-rmse:0.48427
[27]	train-rmse:0.47692	val-rmse:0.44137
[28]	train-rmse:0.4328

In [ ]:
import pickle

# Define the file path to save the model
file_path = '/content/xgboost.pkl'

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    # Use Pickle to serialize and save the XGBoost model
    pickle.dump(bst, file)

# Print a message indicating that the model has been saved
print(f"XGBoost model has been saved to {file_path}")


In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for the test set
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.title('Test Set - Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

# Residual plot for the test set
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.title('Test Set - Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.show()



##References

EDA: https://www.kaggle.com/code/arashnic/max-planck-weather-data-eda

https://seaborn.pydata.org/tutorial.html

https://scikit-learn.org/stable/

https://plotly.com/python/

https://xgboost.readthedocs.io/en/latest/

https://pytorch.org/docs/stable/index.html

https://scikit-learn.org/stable/documentation.html

https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html



